In [20]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn import metrics

In [21]:
def rfimp(feats,pnr):
    
    r = res[feats].copy()
    pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])

    ps = pnr.loc[pnr['Gen'] == 1]
    ns = pnr.loc[pnr['Gen'] == 0]

    pnrs = pd.concat([ps,ns])
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 15, criterion = 'entropy')
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    dunnowhat = metrics.accuracy_score(y_test, y_pred)
    cv_bin = cross_val_score(classifier, X, y, cv=5, scoring='f1')
    cv_mac = cross_val_score(classifier, X, y, cv=5, scoring='f1_macro')
    f1bin = f1_score(y_test, y_pred, average='binary')
    f1mac = f1_score(y_test, y_pred, average='macro')
    return cv_bin, cv_mac, f1bin, f1mac, dunnowhat

In [8]:
aclist = []

for i in range(20):
    fmac = whole(pnr)
    aclist.append(fmac.mean())
    print(fmac.mean(), fmac.std() *2)

print(np.mean(aclist))

0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653657


Selection using SelectFromModel and RFECV 

In [40]:
#selecting from res 340
pn = pd.read_csv('pndf.csv')
res = pd.read_csv('res 340.csv')
pn.loc[pn['Gen'] == 'ssRNA(+)', 'Gen'] = 0
pn.loc[pn['Gen'] == 'ssRNA(-)', 'Gen'] = 1

fts = fdf.sort_values(by='final', ascending = False)[:50].index
r = res[fts].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des', 'Len'])

ps = pnr.loc[pnr['Gen'] == 1]
ns = pnr.loc[pnr['Gen'] == 0]

pnrs = pd.concat([ps,ns], axis = 0).reset_index(drop=True)

#Splitting features and labels
X = pnrs.iloc[:, 3:].values.astype(float)
y = pnrs.iloc[:, 1].values.astype(float)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV
#classifier = RandomForestClassifier(n_estimators = 20, criterion = 'entropy')
#classifier.fit(X_train, y_train)

#feature selection 1
selector1 = SelectFromModel(RandomForestClassifier(n_estimators = 20))
selector1.fit(X_train, y_train)
#feature selection 2
selector2 = RFECV(RandomForestClassifier(n_estimators = 20), step=1, cv=5)
selector2 = selector2.fit(X_train, y_train)

# Predicting the Test set results
#y_pred = sel.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(y_test, y_pred)

cv_bin = cross_val_score(classifier, X_train, y_train, cv=5, scoring='f1')
cv_mac = cross_val_score(classifier, X_train, y_train, cv=5, scoring='f1_macro')

f1bin = f1_score(y_test, y_pred, average='binary')
f1mac = f1_score(y_test, y_pred, average='macro')

NameError: name 'classifier' is not defined

In [44]:
pnrs.iloc[:, 3:].columns[(sel.get_support())]

Index(['Len', 'ATCA', 'ATCT', 'GCGT', 'TGGT', 'ATC', 'GTG', 'GATC', 'TATC',
       'TCA', 'AATC', 'TCAT', 'TCAG'],
      dtype='object')

In [45]:
pnrs.iloc[:, 3:].columns[(selector2.get_support())]

Index(['Len', 'ATCA', 'ATCT', 'GCGT', 'TGGT', 'ATC', 'GTG', 'GATC', 'TATC',
       'TCA', 'GAT', 'CGCT', 'AGAT', 'CGT', 'TCAG', 'AGA', 'CACG'],
      dtype='object')

selecting from lenc and tp

In [7]:
pnrs.iloc[:, 5]

0       1
1       1
2       1
3       1
4       1
       ..
3140    0
3141    0
3142    0
3143    0
3144    0
Name: Gen, Length: 3145, dtype: object

In [3]:
fdf = pd.read_csv('../fdf2 win5.csv', index_col = 'Unnamed: 0')

In [71]:
facs = fdf.sort_values(by= 'final', ascending = False)[:120].index

In [11]:
#selecting for tps and res 340
pn = pd.read_csv('pndf.csv')
res = pd.read_csv('../win5.csv')
pn.loc[pn['Gen'] == 'ssRNA(+)', 'Gen'] = 0
pn.loc[pn['Gen'] == 'ssRNA(-)', 'Gen'] = 1

fts = fdf.sort_values(by='result', ascending = False)[:120].index
r = res[fts].copy()
pnr = pd.concat([pn, r], axis = 1)

ps = pnr.loc[pnr['Gen'] == 1]
ns = pnr.loc[pnr['Gen'] == 0]

pnrs = pd.concat([ps,ns], axis = 0).reset_index(drop=True)

def tpfeatsel(pnrs):
    
    #Splitting features and labels
    X = pnrs.iloc[:, -120:].values.astype(float)
    y = pnrs.iloc[:, 5].values.astype(float)

    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_selection import SelectFromModel
    from sklearn.feature_selection import RFECV
    classifier = RandomForestClassifier(n_estimators = 20)
    #classifier.fit(X_train, y_train)

    #feature selection 1
    selector1 = SelectFromModel(RandomForestClassifier(n_estimators = 20))
    selector1.fit(X_train, y_train)
    #feature selection 2
    selector2 = RFECV(RandomForestClassifier(n_estimators = 20), step=1, cv=5)
    selector2 = selector2.fit(X_train, y_train)

    #Predicting the Test set results
    #y_pred = classifier.predict(X_test)

    #Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    #cm = confusion_matrix(y_test, y_pred)

    cv_bin = cross_val_score(classifier, X_train, y_train, cv=5, scoring='f1')
    cv_mac = cross_val_score(classifier, X_train, y_train, cv=5, scoring='f1_macro')

    #f1bin = f1_score(y_test, y_pred, average='binary')
    #f1mac = f1_score(y_test, y_pred, average='macro')
    
    return list(pnrs.iloc[:,-120:].columns[(selector2.get_support())]) + \
                list(pnrs.iloc[:,-120:].columns[(selector1.get_support())]), cv_mac

In [12]:
tfdf = pd.DataFrame(index = pnrs.iloc[:,-120:].columns, columns = ['count', 'mac'])
tfdf['count'] = 0
tfdf['mac'] = 0

for i in range(20):
    lit, mac = tpfeatsel(pnrs)
    for f in lit:
        tfdf.loc[f,'count'] = tfdf.loc[f,'count'] + 1
        tfdf.loc[f,'mac'] = tfdf.loc[f,'mac'] + mac.mean()
        
    print('%d iterations remain.' % (19 - i))

19 iterations remain.
18 iterations remain.
17 iterations remain.
16 iterations remain.
15 iterations remain.
14 iterations remain.
13 iterations remain.
12 iterations remain.
11 iterations remain.
10 iterations remain.
9 iterations remain.
8 iterations remain.
7 iterations remain.
6 iterations remain.
5 iterations remain.
4 iterations remain.
3 iterations remain.
2 iterations remain.
1 iterations remain.
0 iterations remain.


In [18]:
tfdf.sort_values(by = 'count', ascending = False)[:20].index

Index(['AGATC', 'GGATC', 'GCGTT', 'GATCA', 'ACGTG', 'ATCAG', 'GATCT', 'ATCAT',
       'CGCGT', 'ATCTC', 'ATATC', 'GTGGT', 'TCAGA', 'TGGTT', 'GCGTA', 'CACGT',
       'TCATC', 'AGCGT', 'CGCTT', 'CGTGT'],
      dtype='object')

In [88]:
tfdf.sort_values(by = 'count', ascending = False).index[:12]

Index(['ATCA', 'ATC', 'ATCT', 'TATC', 'GATC', 'TCAG', 'TGGT', 'CGT', 'GCGT',
       'CAGA', 'TCA', 'TCAT'],
      dtype='object')

In [15]:
tfdf['f1mac'] = tfdf['mac']/tfdf['count']

In [40]:
tfdf.sort_values(by = 'count', ascending = False).index[:9]

Index(['T to G', 'G to A', 'T to T', 'T to C', 'C to G', 'G to T', 'A to T',
       'G to C', 'A to C'],
      dtype='object')

In [23]:
pnrs.iloc[:,-16:].columns[(selector2.get_support())]

Index(['A to A', 'A to T', 'A to G', 'A to C', 'T to A', 'T to T', 'T to G',
       'T to C', 'G to A', 'G to T', 'G to G', 'G to C', 'C to G'],
      dtype='object')

In [22]:
pnrs.iloc[:,-16:].columns[(selector1.get_support())]

Index(['A to T', 'T to T', 'T to G', 'T to C', 'G to A', 'G to T', 'G to C',
       'C to G'],
      dtype='object')